# Тестовое задание, Рогачев Максим Дмитриевич, 12.09.2023

In [210]:
import pandas as pd

import warnings
warnings.filterwarnings('ignore')

## Часть 1. Маржинальная ставка

Скрипт get_borrow_interest.py выгружает данные о маржинальной процентной ставке для каждого инструмента, доступного для торговли. Так как к API ключу имеет только мой IP адрес(иначе нельзя получить доступ к маржинальным данным), и из файла config.ini удалён секретный ключ, то вы не сможете его запустить. С учётом того, что ставка меняется в течение дня непредсказуемое число раз, данные требуют предобработки.

In [211]:
borrow_interest = pd.read_csv('borrow_interest_raw.csv')

Сейчас данные выглядят так: для каждого тикера выгружены все изменения ставки в пределах суток 11.09.2023 03:00:00 - 12.09.2023 03:00:00. Ставка меняется неограниченное число раз, поэтому для каждого инструмента может быть несколько записей. Моя задача - рассчитать суммарный dailyInterestRate для суток 11.09.2023 03:00:00 - 12.09.2023 03:00:00 с учётом изменения ставки. 

In [212]:
borrow_interest

,asset,timestamp,dailyInterestRate,vipLevel
0,BTC,1694534400000,0.000043,1
1,BTC,1694520000000,0.000042,1
2,BTC,1694516400000,0.000043,1
3,BTC,1694502000000,0.000042,1
4,BTC,1694498400000,0.000041,1
...,...,...,...,...
6115,CYBER,1694404800000,0.000655,1
6116,CYBER,1694401200000,0.000524,1
6117,CYBER,1694397600000,0.000419,1
6118,CYBER,1694394000000,0.000457,1


Удаляем все записи, выходящие за пределы суток:

In [213]:
borrow_interest = borrow_interest[borrow_interest['timestamp'] >= 1694401200000]
borrow_interest = borrow_interest[borrow_interest['timestamp'] <= 1694487600000]

Добавляем для каждого тикера запись с пороговыми значениями времени, это нужно для дальнейшего расчета:

In [214]:
temp = borrow_interest.groupby(['asset'])['dailyInterestRate'].last()

for asset in borrow_interest['asset'].unique():
    borrow_interest = borrow_interest.append({'asset': asset, 'timestamp': 1694401200000, 'dailyInterestRate': temp[asset], 'vipLevel': 1}, ignore_index=True)
    borrow_interest = borrow_interest.append({'asset': asset, 'timestamp': 1694487600000, 'dailyInterestRate': 0, 'vipLevel': 1}, ignore_index=True)

borrow_interest = borrow_interest.drop_duplicates(['asset', 'timestamp'])   
borrow_interest.loc[borrow_interest['timestamp'] == 1694487600000, 'dailyInterestRate'] = 0 

Сортируем записи по тикеру и времени изменения ставки:

In [215]:
borrow_interest = borrow_interest.sort_values(['asset', 'timestamp'], ascending=(False, False))

Для примера, так сейчас выглядят данные по биткоину: первая запись привязана к дате 12.09.2023 03:00:00, процентная ставка там 0 для удобства расчётов. Последняя запись привязана к дате 11.09.2023 03:00:00 и ставка там 0.000041(обратите внимание, что ставка на самом деле представлена более точной дробью, но из - за особенностей вывода отображается неполное число). Далее необходимо рассчитать сколько часов конкретная ставка имела место, затем умножить полученное количество часов на ставку, поделив на 24. В итоге, если провести такие вычисления для каждой ставки внутри одного токена, то мы получим итоговую дневную ставку.

In [216]:
borrow_interest[borrow_interest['asset'] == 'BTC']

,asset,timestamp,dailyInterestRate,vipLevel
3647,BTC,1694487600000,0.000000,1
0,BTC,1694476800000,0.000041,1
1,BTC,1694455200000,0.000041,1
2,BTC,1694448000000,0.000041,1
3646,BTC,1694401200000,0.000041,1


Здесь проводятся вышеописанные вычисления. Для каждой ставки рассчитываем количество часов, перемножаем часы со ставками и складываем, все внутри одного тикера. В этом же разделе переводим ставку в проценты:

In [217]:
borrow_interest['hours'] = (borrow_interest['timestamp'].shift(1) - borrow_interest['timestamp']) // 3600000
borrow_interest['hours*rate'] = borrow_interest['hours'] * borrow_interest['dailyInterestRate']
borrow_interest_day = borrow_interest.groupby(['asset'])['hours*rate'].sum() / 24 * 100

In [218]:
borrow_interest_day_df = borrow_interest_day.to_frame(name='bi_percents')
borrow_interest_day_df.reset_index(inplace=True)
borrow_interest_day_df = borrow_interest_day_df.rename(columns={'asset': 'symbol'})

В конечном итоге имеем такую таблицу. Для каждого инструмента в столбце *bi_percents* указана итоговая ставка в процентах с учётом её изменения:

In [219]:
borrow_interest_day_df

,symbol,bi_percents
0,1INCH,0.027572
1,AAVE,0.019641
2,ACH,0.024948
3,ADA,0.018118
4,AGIX,0.012944
...,...,...
243,YGG,0.094073
244,ZEC,0.007966
245,ZEN,0.012608
246,ZIL,0.021065


## Часть 2. Ставка финансирования

Скрипт get_fundig_fee.py выгружает данные о ставке финансирования для каждого доступного инструмента в пределах суток 11.09.2023 03:00:00 - 12.09.2023 03:00:00. Так как он не привязан к API, вы можете запустить его самостоятельно.  

Данные выглядят так: для каждого инструмента указывается три ставки в столбце *fundingRate*.

In [220]:
funding_fee = pd.read_csv('funding_fees_raw.csv')
funding_fee

,symbol,fundingTime,fundingRate
0,SUSHIUSDT,1694390400000,-0.000073
1,SUSHIUSDT,1694419200000,-0.000084
2,SUSHIUSDT,1694448000001,0.000027
3,BTSUSDT,1694390400000,0.000100
4,BTSUSDT,1694419200000,0.000100
...,...,...,...
626,CTSIUSDT,1694419200000,-0.000097
627,CTSIUSDT,1694448000001,0.000100
628,ACHUSDT,1694390400000,-0.000420
629,ACHUSDT,1694419200000,0.000100


Так как здесь ставка обновляется фиксированное число раз в день и в равные промежутки, то достаточно просто сложить ставки, результат взять по модулю и перевести в проценты:

In [221]:
funding_fee_day = funding_fee.groupby(['symbol'])['fundingRate'].sum().abs() * 100

In [222]:
funding_fee_day_df = funding_fee_day.to_frame(name='ff_percents')
funding_fee_day_df.reset_index(inplace=True)
funding_fee_day_df['symbol'] = funding_fee_day_df['symbol'].apply(lambda x: x[0:-4])

В итоге получаем таблицу, в которой каждому тикеры ставится в соответствие процент, который мы заработали бы за сутки:

In [223]:
funding_fee_day_df

,symbol,ff_percents
0,1000FLOKI,0.030000
1,1000LUNC,0.004436
2,1000PEPE,0.017130
3,1000SHIB,0.003202
4,1000XEC,0.004733
...,...,...
204,YGG,0.015381
205,ZEC,0.018053
206,ZEN,0.007725
207,ZIL,0.011759


## Часть 3. Поиск потенциально прибыльных монет

Следующий шаг - объединить таблицы по тикерам. В конечном итоге остаётся 180 тикеров, для каждого из которых известны дневная процентная ставка по маржинальной сделке и дневной процент финансирования по фьючерсной сделке:

In [224]:
all_data = borrow_interest_day_df.merge(funding_fee_day_df, on=['symbol'])
all_data

,symbol,bi_percents,ff_percents
0,1INCH,0.027572,0.021983
1,AAVE,0.019641,0.001869
2,ACH,0.024948,0.021985
3,ADA,0.018118,0.014414
4,AGIX,0.012944,0.017848
...,...,...,...
175,YGG,0.094073,0.015381
176,ZEC,0.007966,0.018053
177,ZEN,0.012608,0.007725
178,ZIL,0.021065,0.011759


Оставляем только те инструменты, у которых funding_fee превышает borrow_interest в рамках суток и рассчитываем дневной заработок в процентах, путем вычитания маржинальной ставки из ставки финансирования:

In [225]:
all_data = all_data[all_data['ff_percents'] > all_data['bi_percents']]
all_data['income_percent'] = all_data['ff_percents'] - all_data['bi_percents']

В итоге для каждого инструмента получаем дневной заработок в столбце *income_percent*:

In [226]:
all_data.head()

,symbol,bi_percents,ff_percents,income_percent
4,AGIX,0.012944,0.017848,0.004904
6,ALGO,0.021427,0.056837,0.035410
7,ALICE,0.014705,0.024405,0.009700
13,APT,0.039691,0.058251,0.018560
14,AR,0.005406,0.011521,0.006115


## Часть 4. Расчёт времени покрытия комиссий

Binance взимает с тейкеров со статусом VIP1 следующие комиссии:

In [227]:
futures_fee_taker = 0.04
margin_fee_taker = 0.1

С учётом этого добавляем в таблицу столбец, отражающий количество дней, необходимое для покрытия двух комиссий(покупка и продажа) в фьючерсах и марже:

In [228]:
all_data['Days_until_fee_is_covered'] = 2*(futures_fee_taker + margin_fee_taker) / all_data['income_percent']
all_data = all_data.sort_values(['Days_until_fee_is_covered'], ascending=True).reset_index(drop=True)

В столбце *Days_until_fee_is_covered* хранится количество дней, необходимое для покрытия комиссий:

In [229]:
all_data.head()

,symbol,bi_percents,ff_percents,income_percent,Days_until_fee_is_covered
0,BLZ,1.516632,7.644136,6.127504,0.045696
1,TRB,0.266503,4.445641,4.179138,0.066999
2,ENJ,0.062530,0.238983,0.176453,1.586821
3,DOT,0.020816,0.069331,0.048515,5.771411
4,ALGO,0.021427,0.056837,0.035410,7.907473


Полная таблица для ознакомления:

In [230]:
all_data

,symbol,bi_percents,ff_percents,income_percent,Days_until_fee_is_covered
0,BLZ,1.516632,7.644136,6.127504,0.045696
1,TRB,0.266503,4.445641,4.179138,0.066999
2,ENJ,0.062530,0.238983,0.176453,1.586821
3,DOT,0.020816,0.069331,0.048515,5.771411
4,ALGO,0.021427,0.056837,0.035410,7.907473
5,XTZ,0.037354,0.063233,0.025879,10.819671
6,AVAX,0.018867,0.043816,0.024949,11.223045
7,ATA,0.006326,0.030000,0.023674,11.827404
8,CKB,0.008129,0.030000,0.021871,12.802414
9,RVN,0.007414,0.026833,0.019419,14.418837


Для нескольких лидеров рассчитаем этот параметр в минутах:

In [231]:
all_data['Minuts_until_fee_is_covered'] = all_data['Days_until_fee_is_covered']*24*60
all_data.head(2)

,symbol,bi_percents,ff_percents,income_percent,Days_until_fee_is_covered,Minuts_until_fee_is_covered
0,BLZ,1.516632,7.644136,6.127504,0.045696,65.801668
1,TRB,0.266503,4.445641,4.179138,0.066999,96.479223


## Вывод

Самыми прибыльными оказались BLZ и TRB, которые покроют комиссии за 65 и 96 минут соответственно. 